In [1]:
import pandas as pd
import sys
import os
import inspect

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
output_llm_folder_path = ('f6_llm_tst_data/')
output_evaluation_folder_path = 'f7_processed_llm_tst_data/'

prompting_strategies = ['Prompts_NoContext', 'Prompts_Context']


for prompting in prompting_strategies:
    output_df = pd.DataFrame()
    
    for subdir, dirs, files in os.walk(output_llm_folder_path+prompting):
        file_id = 0
        for file in files:
            filepath = subdir + os.sep + file
            # filter out relevant info
            if filepath.endswith("output.csv"):
                # print(file_id)
                # print(filepath)
                temp_details = file.split('_')
                temp_output = pd.read_csv(filepath)[['original', 'rewritten_sentence']]
                temp_output['fileID'] = file_id
                temp_output['user'] = temp_details[1]
                temp_output['promptID'] = temp_details[3] if prompting == 'Prompts_NoContext' else temp_details[4]
                temp_output['model'] = temp_details[5] if prompting == 'Prompts_NoContext' else temp_details[6]
                temp_output['shots'] = temp_details[7] if prompting == 'Prompts_NoContext' else temp_details[8]
                temp_output['runID'] = temp_details[10] if prompting == 'Prompts_NoContext' else temp_details[11]
                
                # concat temp dataframe
                output_df = pd.concat([output_df, temp_output], ignore_index=True)
                file_id = file_id + 1
            
        # reorder columns
        output_df = output_df[['fileID', 'user', 'promptID', 'model', 'shots', 'runID', 'original', 'rewritten_sentence']]
    
        # save to csv
        output_df.to_csv(output_evaluation_folder_path + prompting + '_complete_output.csv', index=False)
